# scVI

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>).
- **Affiliation:** Helmholtz Munich, Institute of Computational Biology (ICB), Talavera-López Lab
- **Date of Creation:** 05.01.2023
- **Date of Last Modification:** 19.08.2023

- The scVI source code is available at https://github.com/scverse/scvi-tools.
- The corresponding publication is "Lopez, R., Regier, J., Cole, M. B., Jordan, M. I. & Yosef, N. Deep generative modeling for single-cell transcriptomics. Nat. Methods 15, 1053–1058 (2018)".
- The workflow of this notebook follows the tutorial from https://docs.scvi-tools.org/en/stable/tutorials/notebooks/harmonization.html.
- The authors use raw counts as input to scVI. Therefore, we also use raw counts (stored in adata.layers["counts"]).

## 1. Setup

### 1.1 Import Libraries

In [1]:
import gc
import os
import time
from datetime import datetime

import anndata as ad
import scvi
import scanpy as sc
import scipy.sparse as sp
import squidpy as sq
import matplotlib.pyplot as plt
import numpy as np

[rank: 0] Global seed set to 0
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


### 1.2 Define Parameters

In [2]:
model_name = "scvi"
latent_key = f"{model_name}_latent"
mapping_entity_key = "reference"
condition_key = "batch"
counts_key = "counts"
leiden_resolution = 0.5 # used for Leiden clustering of latent space
random_seed = 0 # used for Leiden clustering

### 1.3 Run Notebook Setup

In [3]:
sc.set_figure_params(figsize=(6, 6))

In [4]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

### 1.4 Configure Paths and Directories

In [5]:
srt_data_gold_folder_path = "../../datasets/srt_data/gold/"
srt_data_results_folder_path = "../../datasets/srt_data/results/" 
figure_folder_path = f"../../figures"
benchmarking_folder_path = "../../artifacts/sample_integration_method_benchmarking"

# Create required directories
os.makedirs(srt_data_gold_folder_path, exist_ok=True)
os.makedirs(srt_data_results_folder_path, exist_ok=True)

## 2. scVI Model

### 2.1 Define Training Function

In [6]:
def train_scvi_models(dataset,
                      reference_batches,
                      cell_type_key,
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16],
                      plot_latent_umaps=False):    
    # Create new adata to store results from training runs in storage-efficient way
    if adata_new is None:  
        adata_batch_list = []
        if reference_batches is not None:
            for batch in reference_batches:
                adata_batch = ad.read_h5ad(
                    f"{srt_data_gold_folder_path}/{dataset}_{batch}.h5ad")
                adata_batch.obs[mapping_entity_key] = "reference"
                adata_batch_list.append(adata_batch)
            adata_original = ad.concat(adata_batch_list, join="inner")
        else:
            adata_original = ad.read_h5ad(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

        adata_new = sc.AnnData(sp.csr_matrix(
            (adata_original.shape[0], adata_original.shape[1]),
            dtype=np.float32))
        adata_new.var_names = adata_original.var_names
        adata_new.obs_names = adata_original.obs_names
        adata_new.obs["cell_type"] = adata_original.obs[cell_type_key].values
        adata_new.obsm["spatial"] = adata_original.obsm["spatial"]
        adata_new.obs[condition_key] = adata_original.obs[condition_key]
        adata_new.obs[mapping_entity_key] = adata_original.obs[mapping_entity_key] 
        del(adata_original)
        
    model_seeds = list(range(10))
    for run_number, n_neighbors in zip(np.arange(n_start_run, n_end_run+1), n_neighbor_list):
        # n_neighbors is here only used for the latent neighbor graph construction used for
        # UMAP generation and clustering as scVI is not a spatial method
        
        # Load data
        adata_batch_list = []
        if reference_batches is not None:
            for batch in reference_batches:
                print(f"Processing batch {batch}...")
                print("Loading data...")
                adata_batch = ad.read_h5ad(
                    f"{srt_data_gold_folder_path}/{dataset}_{batch}.h5ad")
                adata_batch_list.append(adata_batch)
            adata = ad.concat(adata_batch_list, join="inner")
        else:
            adata = ad.read_h5ad(f"{srt_data_gold_folder_path}/{dataset}.h5ad")
        
        start_time = time.time()
        
        scvi.settings.seed = model_seeds[run_number-1]

        # Setup adata
        scvi.model.SCVI.setup_anndata(adata,
                                      layer=counts_key,
                                      batch_key=condition_key)

        # Initialize model
        # Use hyperparams that provenly work well on integration tasks
        model = scvi.model.SCVI(adata,
                                n_layers=2,
                                n_latent=30,
                                gene_likelihood="nb")

        # Train model
        model.train()

        # Store latent representation
        adata.obsm[latent_key] = model.get_latent_representation()
        
        # Measure time for model training
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Duration of model training in run {run_number}: "
              f"{int(hours)} hours, {int(minutes)} minutes and {int(seconds)} seconds.")
        adata_new.uns[f"{model_name}_model_training_duration_run{run_number}"] = (
            elapsed_time)
        
        if plot_latent_umaps:
            # Configure figure folder path
            dataset_figure_folder_path = f"{figure_folder_path}/{dataset}/sample_integration_method_benchmarking/" \
                                         f"{model_name}/{current_timestamp}"
            os.makedirs(dataset_figure_folder_path, exist_ok=True)
    
            # Use scVI latent space for UMAP generation
            sc.pp.neighbors(adata,
                            use_rep=latent_key,
                            n_neighbors=n_neighbors)
            sc.tl.umap(adata)
            fig = sc.pl.umap(adata,
                             color=[cell_type_key],
                             title="Latent Space with Cell Types: scVI",
                             return_fig=True)
            fig.savefig(f"{dataset_figure_folder_path}/latent_{model_name}"
                        f"_cell_types_run{run_number}.png",
                        bbox_inches="tight")

            # Compute latent Leiden clustering
            sc.tl.leiden(adata=adata,
                         resolution=leiden_resolution,
                         random_state=random_seed,
                         key_added=f"latent_{model_name}_leiden_{str(leiden_resolution)}")

            # Create subplot of latent Leiden cluster annotations in physical and latent space
            fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 12))
            title = fig.suptitle(t="Latent and Physical Space with Leiden Clusters: scVI")
            sc.pl.umap(adata=adata,
                       color=[f"latent_{model_name}_leiden_{str(leiden_resolution)}"],
                       title=f"Latent Space with Leiden Clusters",
                       ax=axs[0],
                       show=False)
            sq.pl.spatial_scatter(adata=adata,
                                  color=[f"latent_{model_name}_leiden_{str(leiden_resolution)}"],
                                  title=f"Physical Space with Leiden Clusters",
                                  shape=None,
                                  ax=axs[1])

            # Create and position shared legend
            handles, labels = axs[0].get_legend_handles_labels()
            lgd = fig.legend(handles, labels, bbox_to_anchor=(1.25, 0.9185))
            axs[0].get_legend().remove()
            axs[1].get_legend().remove()

            # Adjust, save and display plot
            plt.subplots_adjust(wspace=0, hspace=0.2)
            fig.savefig(f"{dataset_figure_folder_path}/latent_physical_comparison_"
                        f"{model_name}_run{run_number}.png",
                        bbox_extra_artists=(lgd, title),
                        bbox_inches="tight")
            plt.show()
        
        # Store latent representation
        adata_new.obsm[latent_key + f"_run{run_number}"] = adata.obsm[latent_key]
        
        # Use latent representation for UMAP generation
        sc.pp.neighbors(adata_new,
                        use_rep=f"{latent_key}_run{run_number}",
                        key_added=f"{latent_key}_run{run_number}")
        sc.tl.umap(adata_new,
                   neighbors_key=f"{latent_key}_run{run_number}")
        adata_new.obsm[f"{latent_key}_run{run_number}_X_umap"] = adata_new.obsm["X_umap"]
        del(adata_new.obsm["X_umap"])

        # Store intermediate adata to disk
        adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad")  

        # Free memory
        del(adata)
        del(model)
        gc.collect()
        
    # Store final adata to disk
    adata_new.write(f"{benchmarking_folder_path}/{dataset}_{model_name}.h5ad") 

### 2.2 Train Models on Benchmarking Datasets

In [ ]:
train_scvi_models(dataset="seqfish_mouse_organogenesis",
                  reference_batches=[f"batch{i}" for i in range(1,7)],
                  cell_type_key="celltype_mapped_refined",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_scvi_models(dataset=f"nanostring_cosmx_human_nsclc_subsample_{subsample_pct}pct",
                      reference_batches=[f"batch{i}" for i in range(1,4)],
                      cell_type_key="cell_type",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

/home/aih/sebastian.birk/miniconda3/envs/nichecompass-reproducibility/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Processing batch batch1...
Loading data...
Processing batch batch2...
Loading data...
Processing batch batch3...
Loading data...


/home/aih/sebastian.birk/miniconda3/envs/nichecompass-reproducibility/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
[rank: 0] Global seed set to 0
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/aih/sebastian.birk/miniconda3/envs/nichecompas ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available 

Epoch 57/57: 100%|███████████| 57/57 [05:23<00:00,  5.70s/it, loss=452, v_num=1]

`Trainer.fit` stopped: `max_epochs=57` reached.


...

Epoch 57/57: 100%|███████████| 57/57 [05:24<00:00,  5.69s/it, loss=442, v_num=1]

`Trainer.fit` stopped: `max_epochs=57` reached.


Epoch 57/57: 100%|███████████| 57/57 [05:24<00:00,  5.70s/it, loss=442, v_num=1]
Duration of model training in run 3: 0 hours, 5 minutes and 26 seconds.
Processing batch batch1...
Loading data...
Processing batch batch2...
Loading data...
Processing batch batch3...
Loading data...


/home/aih/sebastian.birk/miniconda3/envs/nichecompass-reproducibility/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
[rank: 0] Global seed set to 3
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/aih/sebastian.birk/miniconda3/envs/nichecompas ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available 

Epoch 34/57:  58%|██████▎    | 33/57 [03:09<02:18,  5.78s/it, loss=449, v_num=1]

In [ ]:
for subsample_pct in [50, 25, 10, 5, 1]:
    train_scvi_models(dataset=f"seqfish_mouse_organogenesis_subsample_{subsample_pct}pct",
                      reference_batches=[f"batch{i}" for i in range(1,7)],
                      cell_type_key="celltype_mapped_refined",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

Processing batch batch1...
Loading data...
Processing batch batch2...
Loading data...
Processing batch batch3...
Loading data...
Processing batch batch4...
Loading data...
Processing batch batch5...
Loading data...
Processing batch batch6...
Loading data...


[rank: 0] Global seed set to 0
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/aih/sebastian.birk/miniconda3/envs/nichecompas ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/aih/sebastian.birk/miniconda3/envs/nichecompas ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 304/304: 100%|████| 304/304 [06:14<00:00,  1.16s/it, loss=258, v_num=1]

`Trainer.fit` stopped: `max_epochs=304` reached.


Epoch 304/304: 100%|████| 304/304 [06:14<00:00,  1.23s/it, loss=258, v_num=1]
Duration of model training in run 1: 0 hours, 6 minutes and 17 seconds.
Processing batch batch1...
Loading data...
Processing batch batch2...
Loading data...
Processing batch batch3...
Loading data...
Processing batch batch4...
Loading data...
Processing batch batch5...
Loading data...
Processing batch batch6...
Loading data...


...

Epoch 304/304: 100%|████| 304/304 [05:51<00:00,  1.16s/it, loss=257, v_num=1]
Duration of model training in run 3: 0 hours, 5 minutes and 52 seconds.
Processing batch batch1...
Loading data...
Processing batch batch2...
Loading data...
Processing batch batch3...
Loading data...
Processing batch batch4...
Loading data...
Processing batch batch5...
Loading data...


[rank: 0] Global seed set to 3


Processing batch batch6...
Loading data...


/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/aih/sebastian.birk/miniconda3/envs/nichecompas ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aih/sebastian.birk/.local/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/aih/sebastian.birk/miniconda3/envs/nichecompas ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 166/304:  54%|██▏ | 165/304 [03:08<02:38,  1.14s/it, loss=255, v_num=1]

In [ ]:
train_scvi_models(dataset="nanostring_cosmx_human_nsclc",
                  reference_batches=[f"batch{i}" for i in range(1, 4)],
                  cell_type_key="cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=8,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])

In [ ]:
for subsample_pct in [1, 5, 10, 25, 50]:
    train_scvi_models(dataset=f"nanostring_cosmx_human_nsclc_{subsample_pct}pct",
                      reference_batches=[f"batch{i}" for i in range(1,4)],
                      cell_type_key="cell_type",
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=8,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16])